In [60]:
import pandas as pd

In [61]:
df = pd.read_csv("spotify_millsongdata.csv")

In [62]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [63]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [64]:
df.shape

(57650, 4)

In [65]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [66]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [67]:
df.head(10)

,artist,song,text
0,OneRepublic,Goodbye Apathy,"I can't sleep, no not like I used to \r\nI ca..."
1,Bob Marley,Donna,"I need you, yes, I need you, Donna) \r\nDonna..."
2,Nick Cave,Are You The One I've Been Waiting For?,"I've felt you coming girl, as you drew near \..."
3,Iron Butterfly,Flowers And Beads,"Girl, I just know I love ya now. \r\nFlowers ..."
4,King Crimson,Prozakc Blues,"Well, I woke up this morning in a cloud of des..."
5,Faces,"You Can Make Me Dance, Sing, Or Anything","Hey baby, I want to say somethin' to ya', list..."
6,Lil Wayne,Anne,"(Out of Sight, \r\nOut of Mind, \r\nOut of T..."
7,Bette Midler,Paradise,"There is a land I know, \r\nWhere lovers go, ..."
8,Underoath,You're So Intricate,Like I said \r\nLeave your baggage at the bac...
9,Kate Bush,Misty,Roll his body. \r\nGive him eyes. \r\nMake h...


In [68]:
df['text'][0]

"I can't sleep, no not like I used to  \r\nI can't breathe in and out like I need to  \r\nIt's breaking ice now to make any movement  \r\nWhat's your vice? You know that mine's the illusion  \r\n  \r\nAll at once  \r\n(As I'm trying)  \r\nI can help you out  \r\n(Just to keep things right)  \r\nI'll be what you need, I'll do anything  \r\n(I kill myself to make everything perfect for you)  \r\nGoodbye apathy, so long apathy  \r\n  \r\nSo don't sit still  \r\nDon't you move away from here  \r\n  \r\nSo goodbye apathy  \r\n(As I'm trying)  \r\nSo long fancy free  \r\n(Just to keep things right)  \r\nGoodbye apathy, I don't wanna be you  \r\n(Kill myself to make everything perfect for you)  \r\n  \r\nI don't walk right, not like I used to  \r\nThere's a jump in my step as I rush to see you  \r\nI could be happy here as long as you're near to me  \r\nAs long as you're close to me  \r\n  \r\nNow that I'm alright  \r\n(As I'm trying)  \r\nI can help you out  \r\n(Just to keep things right)  

In [69]:
# df = df.sample(5000)

In [70]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [71]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [72]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [73]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [75]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [76]:
similarity[0]

array([1.        , 0.04991419, 0.067011  , ..., 0.06317436, 0.05920036,
       0.04929251])

In [77]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [78]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [80]:
recommendation('Goodbye Apathy')

['Story Of My Life',
 '(Everybody) Get Down',
 "Everybody's Trying To Be My Baby",
 'Everybody Talks',
 'Goodbye',
 'Going To A Go-Go',
 "For What It's Worth",
 "All's Quiet On The Eastern Front",
 "Somebody's Watching Me",
 'Day Is Done',
 "Everybody's Gonna Be Happy",
 'Everybody Wants To Rule The World',
 "Everybody's Somebody's Fool",
 'Streamline',
 'Goodbye',
 'Fred Astaire',
 'Goodbye My Friend',
 'No Turning Back',
 'Pleasure, Little Treasure',
 "I've Got A Feeling"]

In [81]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))